## CV11 - OCR 기술의 개요
OCR = Text detection + Text recognition
### Before Deep Learning
OCR의 역사: https://en.wikipedia.org/wiki/Timeline_of_optical_character_recognition  
딥러닝 이전에 이미지 속 문자를 찾아내는 방식: https://arxiv.org/pdf/1811.06193.pdf  
위 논문에서 OCR엔진으로 Tesseract OCR를 사용하였고 1985년부터 개발되어 2006년부턴 구글의 후원을 받고 있다.  
최근엔 LSTM을 적용하는 등 딥러닝 베이스 아키텍처로 발전하고 있다.  
Tesseract OCR을 통한 4단계 과정  
1. Adaptive Thresholding: 입력 영상의 이진화를 통해 흑백처럼 변환
2. Connected Component Analysis: 문자 영역 검출
3. Find Lines and Words: 라인 또는 워드 단위 추출
4. Recognize Word: 문자를 하나씩 인식하고 다시 결합하는 과정

### Text detection
이미지에서 텍스트를 검출할 때는 검출하기 위한 최소 단위를 정해야 한다. (e.g. 문장 단위 혹은 단어, 글자 단위)  
문장, 단어일 경우 길이가 길거나 짧은 문장이나 단어를 찾아내야 한다.  
글자 단위일 경우 모든 텍스트를 찾아낼 수 있지만 이를 다시 단어나 문장으로 맥락에 맞게 이어줘야 한다.  
다양한 Text detection: https://arxiv.org/pdf/1704.03155v2.pdf  
단어 단위의 탐지는 anchor를 정의하고 단어의 유무, bbox의 크기를 추정하여 단어를 찾아낸다. object detection  
글자 단위의 탐지는 글자 영역을 segmentation하는 방법으로 접근한다.  
1. Regression
TextBoxes: https://arxiv.org/pdf/1611.06779.pdf  
이전에는 글자 단위로 인식한 후 결합하는 형태였지만 이 논문에서는 딥러닝을 이용하여 단어 단위로 인식한다. SSD  
보통의 bbox와 달리 텍스트의 특징에 맞게 가로로 긴 bbox를 설정한다. 그리고 촙촙하게 detection하기 위해  
vertical offset를 적용하여 셀과 셀사이에 bbox를 설정한다.  
2. Segmentation
PixelLink: https://arxiv.org/pdf/1801.01315.pdf  
semantic segmentation과 같이 이미지 내에서 객체인 문자과 배경을 분리하고 text가 어느 방향으로 연결되었는지  
같이 학습을 하여 text간 분리 및 연결을 할수 있는 정보를 추가적으로 활용한다.